In [15]:
# %%
import os
import json
import torch
import ast
import numpy as np
from datetime import UTC
from datetime import datetime as dt
import pandas as pd

from hmc.model import ClassificationModel


In [16]:
base_path = "/home/bruno/storage/data/fma/trains/rock_electronic"
test_csv_path = os.path.join(base_path, 'test.csv')
jobs_path = os.path.join(base_path, 'jobs')
model_id = '20240917_114736'



In [17]:
train_path = os.path.join(jobs_path, model_id)
model_path = os.path.join(train_path, 'best_binary.pth')

torch_path = os.path.join(base_path, 'torch')
metadata_path = os.path.join(base_path, "metadata.json")
labels_path = os.path.join(base_path, "labels.json")
testset_path = os.path.join(torch_path, 'test')

In [18]:
df_test = pd.read_csv(test_csv_path)

In [26]:
# Função para converter a string em lista de np.array
def convert_to_list_of_arrays(val):
    # Converte string para uma lista de strings
    list_str = ast.literal_eval(val)
    # Converte cada item da lista para um array NumPy
    return [np.array(item) for item in list_str]


In [27]:
# Aplicar ast.literal_eval para converter as strings em listas
df_test['all_binarized'] = df_test.all_binarized.apply(convert_to_list_of_arrays)

ValueError: malformed node or string on line 1: <ast.Call object at 0x794a68d75a50>

In [25]:
df_test.all_binarized[0]

'['

In [6]:
with open(metadata_path, 'r') as f:
    metadata = json.loads(f.read())
    print(metadata)

with open(labels_path, 'r') as f:
    labels = json.loads(f.read())


{'sequence_size': 1280, 'max_depth': 4, 'levels_size': [2, 29, 15, 2], 'val_path': '/home/bruno/storage/data/fma/trains/rock_electronic/tfrecrods/val', 'train_path': '/home/bruno/storage/data/fma/trains/rock_electronic/tfrecrods/train', 'test_path': '/home/bruno/storage/data/fma/trains/rock_electronic/tfrecrods/test', 'val_torch_path': '/home/bruno/storage/data/fma/trains/rock_electronic/torch/val', 'train_torch_path': '/home/bruno/storage/data/fma/trains/rock_electronic/torch/train', 'test_torch_path': '/home/bruno/storage/data/fma/trains/rock_electronic/torch/test', 'val_csv': '/home/bruno/storage/data/fma/trains/rock_electronic/val.csv', 'train_csv': '/home/bruno/storage/data/fma/trains/rock_electronic/train.csv', 'test_csv': '/home/bruno/storage/data/fma/trains/rock_electronic/test.csv', 'trainset_count': 18209, 'validationset_count': 2007, 'testset_count': 4814}


In [5]:
params = {
    'levels_size': labels['levels_size'],
    'sequence_size': metadata['sequence_size'],
    'dropouts': [0.3, 0.7, 0.7, 0.3]
}


In [6]:

model = ClassificationModel(**params)

# Carregar o state_dict
state_dict = torch.load(model_path)

# Imprimir as chaves do state_dict
print("Chaves do state_dict salvo:")
print(state_dict.keys())

# Imprimir as chaves do modelo
print("Chaves do modelo definido:")
print(model.state_dict().keys())

#model_load = torch.load(binary_model)
model.load_state_dict(state_dict)

/tmp/ipykernel_797680/195874492.py:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(model_path)


Chaves do state_dict salvo:
odict_keys(['levels.0.fc1.weight', 'levels.0.fc1.bias', 'levels.0.fc2.weight', 'levels.0.fc2.bias', 'levels.1.fc1.weight', 'levels.1.fc1.bias', 'levels.1.fc2.weight', 'levels.1.fc2.bias', 'levels.2.fc1.weight', 'levels.2.fc1.bias', 'levels.2.fc2.weight', 'levels.2.fc2.bias', 'levels.3.fc1.weight', 'levels.3.fc1.bias', 'levels.3.fc2.weight', 'levels.3.fc2.bias'])
Chaves do modelo definido:
odict_keys(['levels.0.fc1.weight', 'levels.0.fc1.bias', 'levels.0.fc2.weight', 'levels.0.fc2.bias', 'levels.1.fc1.weight', 'levels.1.fc1.bias', 'levels.1.fc2.weight', 'levels.1.fc2.bias', 'levels.2.fc1.weight', 'levels.2.fc1.bias', 'levels.2.fc2.weight', 'levels.2.fc2.bias', 'levels.3.fc1.weight', 'levels.3.fc1.bias', 'levels.3.fc2.weight', 'levels.3.fc2.bias'])


<All keys matched successfully>

In [7]:
model.to('cuda')

ClassificationModel(
  (levels): ModuleList(
    (0): BuildClassification(
      (fc1): Linear(in_features=1280, out_features=640, bias=True)
      (relu1): ReLU()
      (dropout1): Dropout(p=0.3, inplace=False)
      (fc2): Linear(in_features=640, out_features=2, bias=True)
      (sigmoid): Sigmoid()
    )
    (1): BuildClassification(
      (fc1): Linear(in_features=1282, out_features=641, bias=True)
      (relu1): ReLU()
      (dropout1): Dropout(p=0.7, inplace=False)
      (fc2): Linear(in_features=641, out_features=29, bias=True)
      (sigmoid): Sigmoid()
    )
    (2): BuildClassification(
      (fc1): Linear(in_features=1309, out_features=654, bias=True)
      (relu1): ReLU()
      (dropout1): Dropout(p=0.7, inplace=False)
      (fc2): Linear(in_features=654, out_features=15, bias=True)
      (sigmoid): Sigmoid()
    )
    (3): BuildClassification(
      (fc1): Linear(in_features=1295, out_features=647, bias=True)
      (relu1): ReLU()
      (dropout1): Dropout(p=0.3, inplace=F

In [8]:
# %%
# Salvar os argumentos atuais
time_start = dt.now(UTC)
print("[{}] Predict started at {}".format(id, time_start.strftime("%H:%M:%S")))
print(".......................................")

predict = model.predict(testset_path=testset_path,batch_size=64)

time_end = dt.now(UTC)
time_elapsed = time_end - time_start
print(".......................................")
print("[{}] Predict finished at {} / elapsed time {}s".format(id, time_end.strftime("%H:%M:%S"), time_elapsed.total_seconds()))

[<built-in function id>] Predict started at 15:11:36
.......................................
.......................................
[<built-in function id>] Predict finished at 15:11:36 / elapsed time 0.608668s


In [10]:
predict[2][1]

array([0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0])